In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IPL_Matches_2008_2022.csv')

In [ ]:
data_copy=data.copy()

In [ ]:
data.head()

In [ ]:
teams=pd.concat([data["Team1"],data["Team2"]])
teams=teams.unique()
teams

In [ ]:
cities=data["City"].unique()

In [ ]:
team_city_map = {
    'Sunrisers Hyderabad': 'Hyderabad',
    'Deccan Chargers': 'Hyderabad',
    'Mumbai Indians': 'Mumbai',
    'Gujarat Lions': 'Gujarat',
    'Gujarat Titans': 'Gujarat',
    'Rising Pune Supergiants': 'Pune',
    'Rising Pune Supergiant': 'Pune',
    'Pune Warriors': 'Pune',
    'Royal Challengers Bangalore': 'Bangalore',
    'Royal Challengers Bengaluru': 'Bangalore',
    'Kolkata Knight Riders': 'Kolkata',
    'Delhi Daredevils': 'Delhi',
    'Delhi Capitals': 'Delhi',
    'Kings XI Punjab': 'Punjab',
    'Punjab Kings': 'Punjab',
    'Chennai Super Kings': 'Chennai',
    'Rajasthan Royals': 'Rajasthan',
    'Kochi Tuskers Kerala': 'Kochi',
    'Lucknow Super Giants': 'Lucknow'
}

def team_to_city(df, feature):
    df[feature] = df[feature].map(team_city_map)
    return df

def map_team_city(data):
    data = data.copy()
    for feature in ['Team1', 'Team2', 'TossWinner', 'WinningTeam']:
        if feature in data.columns:
            data = team_to_city(data, feature)
    return data

data = map_team_city(data)

In [ ]:
data.head()

In [ ]:
data=data.drop(["ID","SuperOver"],axis=1)
data.head()

In [ ]:
data['Season'] = pd.to_datetime(data['Date']).dt.year
data['dayofyear'] = pd.to_datetime(data['Date']).dt.dayofyear
data['dayofweek'] = pd.to_datetime(data['Date']).dt.dayofweek

In [ ]:
from collections import defaultdict
import ast

# Create a dictionary to count the player_of_match occurrences
player_scores = defaultdict(int)
# Count the number of times each player has been the player_of_match
for player in data['Player_of_Match']:
    player_scores[player] += 1

# Convert Team1Players and Team2Players from string representation of list to actual list
data['Team1Players'] = data['Team1Players'].apply(ast.literal_eval)
data['Team2Players'] = data['Team2Players'].apply(ast.literal_eval)


# Display the player_of_match_counts
player_scores

In [ ]:
players_set = set()
for i in range(data.shape[0]):
    for j in data.loc[i, 'Team1Players']:
        players_set.add(j)
    for k in data.loc[i, 'Team2Players']:
        players_set.add(k)

players_set

In [ ]:
def calculate_team_score(player_list):
    return sum(player_scores[player] for player in player_list)


def add_team_scores(df):
    data = df.copy()
    data['Team1Score'] = data['Team1Players'].apply(calculate_team_score)
    data['Team2Score'] = data['Team2Players'].apply(calculate_team_score)
    return data

data = add_team_scores(data)

In [ ]:
data.head()

In [ ]:
data = data.drop(columns=['MatchNumber'])

In [ ]:
teams=pd.concat([data["Team1"],data["Team2"], data['City']]).unique()
teams

In [ ]:
data.isna().sum()

In [ ]:
data=data.drop(columns=["method"],axis=1)

In [ ]:
data['City'] = data['City'].fillna('Mumbai')

In [ ]:
data=data.dropna()

In [ ]:
data=data.drop(columns=['Team1Players', 'Team2Players'])

In [ ]:
data['team1_home'] = False
data['team2_home'] = False
data.loc[data['City'] == data['Team1'], 'team1_home'] = True
data.loc[data['City'] == data['Team2'], 'team2_home'] = True

In [ ]:
teams_margin_score={}
matches_played={}
matches_won={}
toss_win_count={}
toss_win_match_win={}
avg_win_margin={}
avg_win_rate={}

for index, row in data.iterrows():
    if row["Team1"] not in teams_margin_score:
        teams_margin_score[row["Team1"]]=0
    if row["Team2"] not in teams_margin_score:
        teams_margin_score[row["Team2"]]=0
    if row["Team1"] not in matches_played:
        matches_played[row["Team1"]]=0
    if row["Team2"] not in matches_played:
        matches_played[row["Team2"]]=0
    if row["Team1"] not in matches_won:
        matches_won[row["Team1"]]=0
    if row["Team2"] not in matches_won:
        matches_won[row["Team2"]]=0
    if row["TossWinner"] not in toss_win_count:
        toss_win_count[row["TossWinner"]]=0
    if row["TossWinner"] not in toss_win_match_win:
        toss_win_match_win[row["TossWinner"]]=0

    teams_margin_score[row["WinningTeam"]]+=row["Margin"]
    matches_played[row["Team1"]]+=1
    matches_played[row["Team2"]]+=1
    if row["WinningTeam"]==row["Team1"]:
        matches_won[row["Team1"]]+=1
    else:
        matches_won[row["Team2"]]+=1
    if row["TossWinner"]==row["WinningTeam"]:
        toss_win_match_win[row["TossWinner"]]+=1
    toss_win_count[row["TossWinner"]]+=1

for i in teams_margin_score:
    avg_win_margin[i]=np.round(teams_margin_score[i]/matches_played[i],2)
    # print(i,avg_win_margin[i])
for i in matches_played:
    avg_win_rate[i]=np.round(matches_won[i]/matches_played[i],2)
    # print(i,avg_win_rate[i])

data["team1_avg_win_margin"]=data["Team1"].map(avg_win_margin)
data["team2_avg_win_margin"]=data["Team2"].map(avg_win_margin)
data["team1_avg_win_rate"]=data["Team1"].map(avg_win_rate)
data["team2_avg_win_rate"]=data["Team2"].map(avg_win_rate)


In [ ]:
data.head()

In [ ]:
data.dtypes[data.dtypes == 'object']

In [ ]:
umpires=pd.concat([data["Umpire1"],data["Umpire2"]]).unique()

In [ ]:
teams_encoder=LabelEncoder()
umpires_encoder=LabelEncoder()
venue_encoder=LabelEncoder()

teams_encoder.fit(teams)
umpires_encoder.fit(umpires)

data['Venue'] = venue_encoder.fit_transform(data['Venue'])

data["City"]=teams_encoder.transform(data["City"])
data["Team1"]=teams_encoder.transform(data["Team1"])
data["Team2"]=teams_encoder.transform(data["Team2"])
data["TossWinner"]=teams_encoder.transform(data["TossWinner"])
data["WinningTeam"]=teams_encoder.transform(data["WinningTeam"])

data=pd.get_dummies(data, columns=['TossDecision'], drop_first=True)

data["Umpire1"]=umpires_encoder.transform(data["Umpire1"])
data["Umpire2"]=umpires_encoder.transform(data["Umpire2"])

In [ ]:
data=data.drop(columns=["Player_of_Match"],axis=1)

In [ ]:
data.head()

In [ ]:
data['team1_wins'] = False
data.loc[data['Team1'] == data['WinningTeam'], 'team1_wins'] = True

In [ ]:
data.head()

In [ ]:
x=data.drop(columns=['team1_wins', "WinningTeam", 'WonBy', 'Date', "Margin"])
y=data["team1_wins"]
x = x.reindex(columns=['Season', 'dayofyear', 'dayofweek', 'Venue', 'City',
                           'Team1', 'Team2', 'team1_avg_win_rate', 'team2_avg_win_rate', 'team1_home', 'team2_home',
                           'team1_avg_win_margin', 'team2_avg_win_margin', 'Team1Score', 'Team2Score',
                           'TossWinner',  'TossDecision_field',
                           'Umpire1', 'Umpire2'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=500)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=300, random_state=99)  # You can adjust n_estimators
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('Classification Report:')
print(classification_report(y_test, y_pred))


In [ ]:
X_train.columns

In [ ]:
toss_decision_list = data_copy["TossDecision"].unique()

In [ ]:
umpires

In [ ]:
X_train.columns

In [ ]:
X_train.head()

In [ ]:
def process_test_data(df):
    df = df.copy()
    df['Season'] = pd.to_datetime(df['Date']).dt.year
    df['dayofyear'] = pd.to_datetime(df['Date']).dt.dayofyear
    df['dayofweek'] = pd.to_datetime(df['Date']).dt.dayofweek
    df = df.drop(columns=['Date'])

    df['team1_home'] = False
    df['team2_home'] = False

    df.loc[df['City'] == df['Team1'], 'team1_home'] = True
    df.loc[df['City'] == df['Team2'], 'team2_home'] = True

    df['team1_avg_win_rate'] = df['Team1'].map(avg_win_rate)
    df['team2_avg_win_rate'] = df['Team2'].map(avg_win_rate)

    df['team1_avg_win_margin'] = df['Team1'].map(avg_win_margin)
    df['team2_avg_win_margin'] = df['Team2'].map(avg_win_margin)

    df = add_team_scores(df)

    df['Team1'] = teams_encoder.transform(df['Team1'])
    df['Team2'] = teams_encoder.transform(df['Team2'])
    df['City'] = teams_encoder.transform(df['City'])
    df['TossWinner'] = teams_encoder.transform(df['TossWinner'])

    df['Venue'] = venue_encoder.transform(df['Venue'])
    df['Umpire1'] = umpires_encoder.transform(df['Umpire1'])
    df['Umpire2'] = umpires_encoder.transform(df['Umpire2'])


    df = df.reindex(columns=['Season', 'dayofyear', 'dayofweek', 'Venue', 'City',
                           'Team1', 'Team2', 'team1_avg_win_rate', 'team2_avg_win_rate', 'team1_home', 'team2_home',
                           'team1_avg_win_margin', 'team2_avg_win_margin', 'Team1Score', 'Team2Score',
                           'TossWinner',  'TossDecision_field',
                           'Umpire1', 'Umpire2'])

    return df

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
from datetime import datetime

teams = avg_win_rate.keys()
venues = data_copy['Venue'].unique()

players1 = ['A Ashish Reddy', 'A Badoni', 'A Chandila', 'A Chopra', 'A Choudhary', 'A Dananjaya', 'A Flintoff', 'A Kumble', 'A Manohar', 'A Mishra', 'A Mithun']
players2 = ['A Mukund', 'A Nehra', 'A Nel', 'A Nortje', 'A Singh', 'A Symonds', 'A Tomar', 'A Uniyal', 'A Zampa', 'AA Bilakhia', 'AA Chavan']


stadium_city_map = {
        'M Chinnaswamy Stadium': 'Bangalore',
        'M Chinnaswamy Stadium, Bengaluru': 'Bangalore',
        'Punjab Cricket Association Stadium, Mohali': 'Mohali',
        'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Mohali',
        'Feroz Shah Kotla': 'Delhi',
        'Arun Jaitley Stadium': 'Delhi',
        'Arun Jaitley Stadium, Delhi': 'Delhi',
        'Wankhede Stadium': 'Mumbai',
        'Wankhede Stadium, Mumbai': 'Mumbai',
        'Eden Gardens': 'Kolkata',
        'Eden Gardens, Kolkata': 'Kolkata',
        'Sawai Mansingh Stadium': 'Jaipur',
        'Sawai Mansingh Stadium, Jaipur': 'Jaipur',
        'Rajiv Gandhi International Stadium, Uppal': 'Hyderabad',
        'Rajiv Gandhi International Stadium': 'Hyderabad',
        'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 'Hyderabad',
        'MA Chidambaram Stadium, Chepauk': 'Chennai',
        'MA Chidambaram Stadium': 'Chennai',
        'MA Chidambaram Stadium, Chepauk, Chennai': 'Chennai',
        'Dr DY Patil Sports Academy': 'Mumbai',
        'Dr DY Patil Sports Academy, Mumbai': 'Mumbai',
        'Newlands': 'Cape Town',
        "St George's Park": 'Port Elizabeth',
        'Kingsmead': 'Durban',
        'SuperSport Park': 'Centurion',
        'Buffalo Park': 'East London',
        'New Wanderers Stadium': 'Johannesburg',
        'De Beers Diamond Oval': 'Kimberley',
        'OUTsurance Oval': 'Bloemfontein',
        'Brabourne Stadium': 'Mumbai',
        'Brabourne Stadium, Mumbai': 'Mumbai',
        'Sardar Patel Stadium, Motera': 'Ahmedabad',
        'Barabati Stadium': 'Cuttack',
        'Vidarbha Cricket Association Stadium, Jamtha': 'Nagpur',
        'Himachal Pradesh Cricket Association Stadium': 'Dharamsala',
        'Himachal Pradesh Cricket Association Stadium, Dharamsala': 'Dharamsala',
        'Nehru Stadium': 'Kochi',
        'Holkar Cricket Stadium': 'Indore',
        'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 'Visakhapatnam',
        'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam': 'Visakhapatnam',
        'Subrata Roy Sahara Stadium': 'Pune',
        'Maharashtra Cricket Association Stadium': 'Pune',
        'Maharashtra Cricket Association Stadium, Pune': 'Pune',
        'Shaheed Veer Narayan Singh International Stadium': 'Raipur',
        'JSCA International Stadium Complex': 'Ranchi',
        'Sheikh Zayed Stadium': 'Abu Dhabi',
        'Sharjah Cricket Stadium': 'Sharjah',
        'Dubai International Cricket Stadium': 'Dubai',
        'Saurashtra Cricket Association Stadium': 'Rajkot',
        'Green Park': 'Kanpur',
        'Narendra Modi Stadium, Ahmedabad': 'Ahmedabad',
        'Zayed Cricket Stadium, Abu Dhabi': 'Abu Dhabi',
        'Eden Gardens, Kolkata': 'Kolkata',
        'Punjab Cricket Association IS Bindra Stadium': 'Mohali',
        'Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh': 'Mohali',
        'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 'Lucknow',
        'Barsapara Cricket Stadium, Guwahati': 'Guwahati',
        'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur': 'Mullanpur'
}

date = widgets.DatePicker(description='Select a date', disabled=False)
venue = widgets.Dropdown(options=venues, description='Venue:')
team1 = widgets.Dropdown(options=teams, description='Team 1:')
team2 = widgets.Dropdown(options=teams, description='Team 2:')
toss_winner = widgets.Dropdown(options=teams, description='Toss Winner:')
team1_players = widgets.SelectMultiple(value=players1, options=players_set, description='Team1 Players')
team2_players = widgets.SelectMultiple(value=players2, options=players_set, description='Team2 Players')
toss_decision_field = widgets.Checkbox(value=False, description='Toss winner chose fielding?')
umpire1 = widgets.Dropdown(options=umpires, description='Umpire 1:')
umpire2 = widgets.Dropdown(options=umpires, description='Umpire 2:')

prediction_result = widgets.Output()

def handle_prediction(btn):
    with prediction_result:
        prediction_result.clear_output()
        if team1.value == team2.value:
            print("Team1 and team2 cannot be the same!")
            return
        if umpire1.value == umpire2.value:
            print("One person can't be both umpires!")
            return

        test_data = {
            'Date': date.value,
            'Venue': venue.value,
            'City': stadium_city_map[venue.value],
            'Team1': team1.value,
            'Team2': team2.value,
            'TossWinner': toss_winner.value,
            'TossDecision_field': toss_decision_field.value,
            'Umpire1': umpire1.value,
            'Umpire2': umpire2.value,
            'Team1Players': team1_players.value,
            'Team2Players': team2_players.value
        }

        if None in test_data.values():
            print("Please fill all fields.")
            return

        test_data_df = pd.Series(test_data).to_frame().T
        test_data_df = process_test_data(test_data_df)
        print(f"Processed Test Data:\n{test_data_df}")
        prediction = model.predict_proba(test_data_df)[0]
        print(f"Prediction: \n{team1.value} - {prediction[1]} \n{team2.value} - {prediction[0]}")

predict_button = widgets.Button(description='Predict')
predict_button.on_click(handle_prediction)

input_form = widgets.VBox([
    date, venue, team1, team2, toss_winner, toss_decision_field, umpire1, umpire2,
    team1_players, team2_players,
    predict_button, prediction_result
])

display(input_form)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [ ]:
et = ExtraTreesClassifier(n_estimators=100, min_samples_split=3, min_samples_leaf=3, max_depth=8, bootstrap=False)

param_dist = {
    'n_estimators': [100, 250, 500, 1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3, 4],
    'bootstrap': [False, True]
}

random_search_et = RandomizedSearchCV(estimator=et, param_distributions=param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=3)

random_search_et.fit(X_train, y_train)

et.fit(X_train, y_train)

In [ ]:
print(f"Best parameters: {random_search_et.best_params_}")
print(f"Best cross-validation score: {random_search_et.best_score_}")

best_model_et = random_search_et.best_estimator_
test_score = best_model_et.score(X_test, y_test)
print(f"Test set score: {test_score}")

In [ ]:
best_model_et = et

In [ ]:
y_pred = best_model_et.predict(X_test)
y_pred_proba = best_model_et.predict_proba(X_test)

In [ ]:
print('ATrS:', best_model_et.score(X_train, y_train))
print('ATeS:', best_model_et.score(X_test, y_test))
print(classification_report(y_test, y_pred))